In [1]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
warnings.filterwarnings("ignore")

In [2]:
with open("./glove.6B.100d.txt", "r", encoding="utf-8") as f:
    for line in f:
        print(line)
        print(len(line.split()))
        break

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

101


In [3]:
def word_to_vec(file_path):
    word_to_vec_dict = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in tqdm(f):
            word = line.split()[0]
            vec = np.asarray(line.split()[1:], dtype=np.float32)
            word_to_vec_dict[word] = vec
    return word_to_vec_dict

word_to_vec_dict = word_to_vec(file_path="./glove.6B.100d.txt")

400000it [00:12, 32445.76it/s]


In [4]:
def get_text_vector(text, word_dict=word_to_vec_dict):
    words = word_tokenize(text.lower())
    word_vectors = []
    for word in words:
        if word.lower() in word_dict:
            word_vectors.append(word_dict[word.lower()])
        else:
            continue
    return np.mean(word_vectors, axis=0)

example_output = get_text_vector(text="The quick brown fox jumps over the lazy dog")
print(example_output)

[ 2.87502278e-02 -6.51510060e-02  3.31091106e-01 -4.65650022e-01
 -1.79282770e-01  5.00148870e-02 -6.74657822e-02  2.93085665e-01
 -4.20005530e-01 -2.09980220e-01  1.18867978e-01  9.45621207e-02
  1.61200345e-01 -7.45707601e-02  2.20839977e-01  1.99698880e-01
  4.31055576e-02 -1.07728437e-01  1.78297553e-02  1.98184326e-01
  2.81547785e-01  2.24947765e-01  3.28172296e-02 -2.90016443e-01
  4.13814664e-01  3.59294564e-01 -2.95193315e-01 -1.26943544e-01
 -7.11237863e-02 -2.37333775e-03 -1.77993402e-02  4.76056635e-01
  1.99063331e-01  7.71860033e-02  1.24151111e-01  4.49099541e-01
 -1.90441445e-01  1.91278994e-01  2.64772445e-01  1.70821011e-01
 -3.66676509e-01 -3.35269988e-01 -1.83722213e-01 -1.31598890e-01
 -1.80422217e-01  9.14322138e-02 -1.41331568e-01 -2.28241235e-01
  1.93425551e-01 -4.44583297e-01 -7.90063441e-02 -2.48832256e-02
  4.97954451e-02  1.00680113e+00 -2.23647222e-01 -2.10082221e+00
  8.81746970e-03  7.20399991e-02  1.16461897e+00  2.81475812e-01
  2.61449553e-02  5.31745

In [5]:
df = pd.read_csv("../HomeWork1/nyt.csv")
print(df.shape)
df.head()

(11519, 2)


,text,label
0,(reuters) - carlos tevez sealed his move to ju...,sports
1,if professional pride and strong defiance can ...,sports
2,"palermo, sicily — roberta vinci beat top-seede...",sports
3,spain's big two soccer teams face a pair of it...,sports
4,the argentine soccer club san lorenzo complete...,sports


In [6]:
def split_df(df, train_size=0.8, val_size=0.1, test_size=0.1, random_state=42):
    train_df, temp_df = train_test_split(df, test_size=(1 - train_size), random_state=random_state)
    val_df, test_df = train_test_split(temp_df, test_size=test_size / (val_size + test_size), random_state=random_state)
    return train_df, val_df, test_df

In [7]:
train_df, val_df, test_df = split_df(df)
print(train_df.shape, val_df.shape, test_df.shape)

(9215, 2) (1152, 2) (1152, 2)


In [8]:
X_train = np.array([get_text_vector(text, word_to_vec_dict) for text in tqdm(train_df["text"].to_list(), total=len(train_df))])
X_val = np.array([get_text_vector(text, word_to_vec_dict) for text in tqdm(val_df["text"].to_list(), total=len(val_df))])
X_test = np.array([get_text_vector(text, word_to_vec_dict) for text in tqdm(test_df["text"].to_list(), total=len(test_df))])
print(X_train.shape, X_val.shape, X_test.shape)

100%|██████████| 1152/1152 [00:04<00:00, 264.03it/s]

(9215, 100) (1152, 100) (1152, 100)


In [9]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train, train_df["label"])

LogisticRegression(max_iter=2000)

In [10]:
test_preds = model.predict(X_test)
accuracy = accuracy_score(test_df["label"], test_preds)
macro_f1 = f1_score(test_df["label"], test_preds, average="macro")
micro_f1 = f1_score(test_df["label"], test_preds, average="micro")

print(f"Accuracy Score: {accuracy}")
print(f"Macro F1-Score: {macro_f1}")
print(f"Micro F1-Score: {micro_f1}")

Accuracy Score: 0.9713541666666666
Macro F1-Score: 0.9246996279169749
Micro F1-Score: 0.9713541666666666
